# Mini poc

## Preprocessing

In [54]:
# à ajouter

## Import

In [55]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

In [67]:
df = pd.read_csv('../datacsv/mini_poc.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16828 entries, 0 to 16827
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Price         16828 non-null  int64 
 1   Manufacturer  16828 non-null  object
 2   Prod. year    16828 non-null  int64 
 3   Mileage       16828 non-null  int64 
dtypes: int64(3), object(1)
memory usage: 526.0+ KB


## Encodage

In [1]:
# Encoder les catégories, normaliser les valeurs numériques !!!
# COncernant le nettoyage, s'assurer que toutes les valeurs à encoder en OneHotEncoder() ait des
# nom de valeurs explicites ex: leather / no_leather au lieu de yes / no 

# Encoder =>
#     X - Gear box type : OneHotEncoder()
#     X - Leather interior : Cas special ?! (remplacer yes - no par leather - no leather) ?!
#     X - Manufacturer : OneHotEncoder()
#     nom doublé, alphabet géorgion - Model : OneHotEncoder()
#     X - Category : OneHotEncoder()
#     X - Fuel type : OneHotEncoder()
#     X - Drive wheels : OneHotEncoder()
#     X - Doors : OneHotEncoder()
#     X - Wheel : OneHotEncoder()
#     X - Color : OneHotEncoder()

# Bloquer les prédictions dont les valeurs dépassent les min et max du dataset ?
# Normaliser =>
#     X - Prod. year : StandardScaler() (explication du choix => GPT)
#     X - Engine volume : StandardScaler()
#     X - Mileage : RobustScaler()
#     X - Cylinders : Vérif si valeurs abérantes même après nottoyage, si oui robustScaler() sinon standardScaler()
#     Levy (imputation) : RobustScaler()
#     Airbags : StandardScaler() ou Robust() ?

# fit_transform de façon grouper selon les transformers puis concat les df


### Manufacturer

In [58]:
enc = OneHotEncoder()

In [59]:
manufacturer = df['Manufacturer']
# Adapter l'encodeur aux données
enc.fit(manufacturer.values.reshape(-1, 1))
# Transformer la colonne 'Manufacturer' à l'aide de l'encodeur
encoded_col = enc.transform(manufacturer.values.reshape(-1, 1))
# Convertir le résultat en un DataFrame
encoded_df = pd.DataFrame(
    encoded_col.toarray(), columns=enc.categories_[0])
# Concaténer le DataFrame encodé avec le DataFrame original
df_encoded = pd.concat([df, encoded_df], axis=1)
# Supprimer la colonne 'marque du véhicule' originale
df_encoded.drop(columns=['Manufacturer'], inplace=True)

## Train test sets

In [60]:
# Séparer les données d'entrée et la variable cible
X = df_encoded.drop('Price', axis=1)
y = df_encoded['Price']

# Diviser les données en ensembles de train et de test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

## Modele entrainement et prédiction

In [61]:
# Instancier un modèle SVR
model = SVR(kernel='rbf')

# Entraîner le modèle sur les données de train
model.fit(X_train, y_train)

# Faire des prédictions sur les données de test
y_pred = model.predict(X_test)

In [62]:
print(len(X_test.index))
print('prediction=>', len(y_pred))

3366
prediction=> 3366


## Métrics

In [63]:
rmse = mean_squared_error(y_test, y_pred, squared=False)
rscore = r2_score(y_test, y_pred)

print('rmse=>', rmse)
print('rscore', rscore)

rmse=> 19169.762122368455
rscore -0.039998905604035295


## Prédiction test

### Affichage d'une ligne de X_test

In [64]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
f= open("outputTestFile.txt", "w")
f.write(str(X_test.iloc[:1]))
f.close

f=open("outputTestFile.txt", "r")
print(f.read())
# print(X_test.iloc[:1])
# print(type(X_test))

       Prod. year  Mileage  ACURA  ALFA ROMEO  ASTON MARTIN  AUDI  BENTLEY  BMW  BUICK  CADILLAC  CHEVROLET  CHRYSLER  CITROEN  DAEWOO  DAIHATSU  DODGE  FERRARI  FIAT  FORD  GAZ  GMC  HAVAL  HONDA  HUMMER  HYUNDAI  INFINITI  ISUZU  JAGUAR  JEEP  KIA  LAMBORGHINI  LANCIA  LAND ROVER  LEXUS  LINCOLN  MASERATI  MAZDA  MERCEDES-BENZ  MERCURY  MINI  MITSUBISHI  MOSKVICH  NISSAN  OPEL  PEUGEOT  PONTIAC  PORSCHE  RENAULT  ROVER  SAAB  SATURN  SCION  SEAT  SKODA  SSANGYONG  SUBARU  SUZUKI  TOYOTA  UAZ  VAZ  VOLKSWAGEN  VOLVO  ZAZ
16273        2014    38624    0.0         0.0           0.0   0.0      0.0  0.0    0.0       0.0        0.0       0.0      0.0     0.0       0.0    0.0      0.0   0.0   0.0  0.0  0.0    0.0    0.0     0.0      1.0       0.0    0.0     0.0   0.0  0.0          0.0     0.0         0.0    0.0      0.0       0.0    0.0            0.0      0.0   0.0         0.0       0.0     0.0   0.0      0.0      0.0      0.0      0.0    0.0   0.0     0.0    0.0   0.0    0.0        0.0   

### Test de prédiction

In [66]:
y_pred_test = model.predict([[1990, 38624, 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]])
print(y_pred_test)

# Possibilité pour créer l'input facilement
# Avoir un df template vide
# Y ajouter les valeurs correspondantes
# Remplir les colonnes vides avec 0

[15479.80092422]


/home/payetquentin/Documents/workspace/briefs/S3_car/backend/.venv/lib/python3.10/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but SVR was fitted with feature names
  warnings.warn(
